# Neural Network from scratch

In [36]:
import numpy as np

In [ ]:
class Network:
    def __init__(self, sizes: list[int]):
        self.num_layers = len(sizes)
        self.sizes = sizes

        # Where y is the size of the previous layer and x the size of the next layer
        self.w = [
            np.random.randn(y, x) for y, x in zip(self.sizes[:-1], self.sizes[1:])
        ]
        self.b = [np.random.randn(y, 1) for y in self.sizes[1:]]

    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        """Sigmoid Activation Function"""
        return 1.0 / (1.0 + np.exp(-z))

    def forward(self, x: np.ndarray):
        for w, b in zip(self.w, self.b):
            print("w ", w.shape)
            print("x ", x.shape)
            z = np.matmul(x, w)
            print(z.shape)
            z += b
            x = self.sigmoid(z)


In [40]:
net = Network([2, 3, 2, 1])


x = np.random.randn(1, 2)
w = np.random.randn(2, 3)

np.matmul(x, w)

array([[ 0.49046532, -0.24526969,  0.30852469]])

In [39]:
x.shape

net.forward(x)

w  (2, 3)
x  (1, 2)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 3)